In [ ]:
import sys
sys.path.insert(1, "thirdparty/OnTheWeightEncodedNIRs/neuralImplicitTools/src/")
sys.path.insert(0, "thirdparty/OnTheWeightEncodedNIRs/neuralImplicitTools/submodules/libigl/python/") 

import geometry as gm
import argparse
import tensorflow as tf
import os

import numpy as np
from tqdm import tqdm

In [ ]:
def loadModel(modelPath, neuralKey=''):
    # LOAD THE MODEL
    #load serialized model
    if neuralKey == '':
        jsonFile = open(modelPath+'.json', 'r')
    else:
        jsonFile = open(neuralKey, 'r')

    sdfModel = tf.keras.models.model_from_json(jsonFile.read())
    jsonFile.close()
    #load weights
    sdfModel.load_weights(modelPath + '.h5')
    #sdfModel.summary()

    return sdfModel

def inferSDF(sdfModel, res):
    # create data sequences
    cubeMarcher = gm.CubeMarcher()
    inferGrid = cubeMarcher.createGrid(res)
    S = sdfModel.predict(inferGrid)
    return -S

In [ ]:
def spin_image(verts, res=(16, 16) ):
    # Normalise model
    verts -= verts.mean(axis=0)
    verts /= verts.std(axis=0)

    img = np.zeros(res, dtype=int)

    radiuses = np.sqrt( np.sum(verts[:, :2] ** 2, axis=1) )
    heights = verts[:, 2]
    
    h_idx = np.clip(np.round((1.5 + heights) / 3 * res[0]), 0, res[0] - 1).astype(int)
    r_idx = np.clip(np.round(radiuses / 2.5 * res[1]), 0, res[1] - 1).astype(int)
    
    for i, j in zip(h_idx, r_idx):
        img[i, j] += 1
    
    return img.reshape(-1).astype(float) / verts.shape[0]

In [63]:
avg_from = 100
res = 16
sampled_points = []
for i in range(res):
    for j in range(res):
        rad = 0.6 * i / res
        x = 2 * np.random.rand(avg_from) - 1
        y = (1 - x ** 2) ** (1/2)
        y_sign = 2 * (np.random.rand(avg_from) > 0.5).astype(float) - 1
        x *= rad
        y *= rad * y_sign
        z = np.ones(avg_from) * 0.5 * (j - int(res / 2)) / res
        
        sampled_points.append(np.vstack([x, z, y]).T)
# sampled_points = tf.convert_to_tensor(np.array(sampled_points).reshape(-1, 3))

In [55]:
root = "/mnt/arteml/classes/6850/data/trained_cars/"
trainedModels = [f.split('.')[0] for f in os.listdir(root) if '.h5' in f]
trainedModels = [os.path.join(root, m) for m in trainedModels]

features = []
for m_path in tqdm(trainedModels):
    model = loadModel(m_path)
    s = model.predict(sampled_points)
    
    f = np.abs(s.reshape(-1, avg_from))
    f = f.mean(axis=1)
    features.append(f)

100%|██████████| 1323/1323 [14:14<00:00,  1.55it/s]


In [61]:
features = np.array(features)
np.save('we_features.npy', features)

In [68]:
features[0].reshape(16, 16)

array([[0.15233085, 0.13301085, 0.10858856, 0.07933905, 0.04647852,
        0.01514442, 0.01609495, 0.04759592, 0.07333713, 0.06957776,
        0.04046546, 0.00943972, 0.02230044, 0.05175715, 0.08424643,
        0.11304448],
       [0.15561554, 0.13375318, 0.10917135, 0.07857325, 0.04651252,
        0.01498318, 0.01611449, 0.04740964, 0.07396949, 0.0685863 ,
        0.03967132, 0.00814161, 0.02297132, 0.05358196, 0.08502713,
        0.11176617],
       [0.1587717 , 0.13499913, 0.10857272, 0.07758784, 0.04663657,
        0.0149223 , 0.01627731, 0.04701911, 0.07523797, 0.06320439,
        0.0332233 , 0.0034949 , 0.02733325, 0.05764891, 0.0884834 ,
        0.11558756],
       [0.16229607, 0.13765045, 0.10872418, 0.07771692, 0.04652677,
        0.01502763, 0.01622921, 0.04690729, 0.06775926, 0.04724872,
        0.02197245, 0.00671269, 0.03593275, 0.06600299, 0.09582727,
        0.12631059],
       [0.16242152, 0.13803166, 0.1114817 , 0.07817166, 0.04652365,
        0.01507896, 0.01622122, 